 ## XFOIL analysis with python UI

## init

In [1]:
import subprocess as sp
import shutil
import sys
import string
import shutil


In [2]:
import os
from pathlib import Path
import sched, time
def boostup(loc):
    p = Path(loc)
    #if the directory is there, delete it
    if p.is_dir():
        shutil.rmtree(loc)
    #when the directory is still ther, sleep
    while p.is_dir():
        time.sleep(2)
    #create da directory
    os.makedirs(loc)
    pass

## file and data mining

In [3]:
def file_mining(file):
    #open file and convert them into list
    f = list(open(file, 'r'))
    #only select those data (need to be test if there is a buy)
    f = f[12:]
    #contains a list contains all the [alpha, CL]
    output = [i.split()[:2] for i in f]
    #since now output value elements are string, we need to con conver them to float
    output = [list(map(float, row)) for row in output]
    return output

def data_mining(data_loc,reys):
    #make the data directory of the airfoil
    airfoils = foil_mining(data_loc)
    for airfoil in airfoils:
        loc = data_loc+airfoil+'/'
        outputs = []
        for rey in reys:
            #get the output for each reynolds number
            output = file_mining(loc+str(rey)+'.txt')
            #put rey data with angle value
            outputs.append([rey,output])
            #example: [rey, [*angle, *cl, *cd]]
        yield airfoil, outputs

In [4]:
from os import listdir
from os.path import isfile, join

def foil_mining(airfoil_loc):
    #get only the files in a directory 
    #if boo=True, only get files, if boo=False
    file_list = [f for f in listdir(airfoil_loc)]
    # get only the airfoil name
    for f in file_list:
        yield f.split('.')[0]

# algorithm

### Fitness function idea
    1: reynolds number score:   rey: weights
                                0.5: 1
                                0.2: 2
    
    2: angle data score:        angle: weights
                                13: 2
                                
                                
    3: sum up those points
    
    
    20% thickness
    more importantly improve lift

In [5]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def data_check(cl):
    if max(cl) < 1.4:
        return True
    else:
        return False
    
def frange(start, finish, interval=1.0):
    #frange starts from small angle to large angle
    assert finish > start, 'frange start with smaller number'
    x = float(start)
    out = [x]
    while x < float(finish):
        x+=interval
        out.append(round(x, 5))
    return out

In [6]:
import numpy as np
def heuristic(mata_data):
    #contains: airfoil info, rey, angles, data
    #for each reynolds number
    final_score = {}
    for airfoil, data in mata_data:
        #data [angle, cl]
        score = 0
        for i in data:
            #get score for the each reynolds number
            angleP = heuristic_angle(i[1])
            #weight them by different reynolds number
            reyP = heuristic_rey(i[0])
            #sum them up
            score += angleP * reyP
            score = float('%.3f' % round(score, 3))
        final_score.update({airfoil: score})
    return final_score

def heuristic_rey(rey):
    #haven't decide the model yet!!!!!
    '''need to discuss this'''
    return 1

def heuristic_angle(datas):
    data = np.transpose(datas)
    if data_check(data[1]):
        #use costumized sigmoid function to get weights
        ## mainly to enhance the importance of the Cls above 8 degree AOT
        weights = 10 * sigmoid((data[0]-3) / 2.5) + 0.01
        #get the score for each angle
        points = data[1] * weights
        #positive all the negative term
        points = np.sqrt(points**2)
        #averge them and times 10
        final_point = np.average(points) * 10
    else:
        pass
    #only output the score up to 3 decimal places
    return final_point

In [7]:
import operator
def sort_airfoil(data, old_champs=[]):
    #sort airfoil according to score from big to small
    candidates = list(data.items())
    candidates += old_champs
    return (sorted(candidates, key=operator.itemgetter(1), reverse=True))

### airfoil generator

In [93]:
def file2cor(files_loc):
    '''this is to get the airfoil cordinates in list'''
    airfoil_cor = {}
    for airfoil, file in files_loc.items():
        #open files
        f = list(open(file, 'r'))
        # get rid of \n term and space
        cordinate = [(i.split('\n')[0]).split(' ')[2:] for i in f]
        cordinate = [list(filter(lambda x: len(x) > 0, cor)) for cor in cordinate]
        cordinate = [list(map(float, row)) for row in cordinate]
        #therefore this becomes [[x1, y1], [x2, y2], [x3, y3] ...]
        
        #convert them into generator
        cordinate = (i for i in cordinate)
        #put the generator back to the dictionary
        airfoil_cor[airfoil] = cordinate
        
    return airfoil_cor

def cor2file(cordinates, loc=''):
    assert type(cordinates) == dict, 'dude, this is dictionary parameter'
    os.makedirs(loc)
    for airfoil, plots in cordinates.items():
        #create a write file
        f = open(loc+'/'+airfoil+'.txt', 'w+')
        #turn every point into string format for joining
        assert type(plots) == list, 'at this stage the plots should be in list formate'
        plots_str = [list(map(str, row)) for row in plots]
        #turned them into right format for writing into files
        write_point = ['   '.join(row) for row in plots_str]
        #write each point's location into the file
        for row in write_point:
            f.write(row+'\n')
        f.close()
    pass

In [117]:
parent = file2cor(winners_loc)
father = list(parent['NACA4421'])
father = list(parent['S823'])
father[0] = father[-1] = [1.0, 0.0]
ind = father.index([0.0, 0.0]) # = 17
father[:ind]
father[ind+1:]


[[0.0, 0.0],
 [0.000174, -0.001931],
 [0.0008, -0.004837],
 [0.002374, -0.009387],
 [0.010193, -0.023971],
 [0.021425, -0.041404],
 [0.03433, -0.059382],
 [0.049121, -0.076692],
 [0.065425, -0.091707],
 [0.084347, -0.102507],
 [0.108058, -0.10978],
 [0.136074, -0.114432],
 [0.168058, -0.116517],
 [0.203782, -0.116043],
 [0.243045, -0.113061],
 [0.285642, -0.107692],
 [0.331343, -0.100136],
 [0.379872, -0.090684],
 [0.430894, -0.079712],
 [0.483993, -0.067673],
 [0.538656, -0.055088],
 [0.594258, -0.042513],
 [0.650057, -0.03052],
 [0.70519, -0.019649],
 [0.758682, -0.010377],
 [0.809471, -0.003071],
 [0.856439, 0.002045],
 [0.89846, 0.004922],
 [0.934449, 0.005687],
 [0.963294, 0.004594],
 [0.9839, 0.00254],
 [0.996032, 0.000732],
 [1.0, 0.0]]

In [8]:
def generator(parents_loc, mutation_intensity):
    #open those locations and get the plots
    parents = file2cor(parents_loc)
    for dad_name, father in parents.items():
        for mom_name, mother in parents.values():
            #when parents not same
            if dad_name != mom_name:
                #papa and mama produce some children
                reproduction(father, mother, mutation_intensity)
    pass

def reproduction(father, mother, mutation_intensity):
    #every airfoil should start and end with [1.0, 0.0]
    father = list(father)
    mother = list(mother)
    'father[0] = father[-1] = [1.0, 0.0]'
    
    #find the split point
    idxF = father.index([0.0, 0.0])
    idxM = mother.index([0.0, 0.0])
    
    #split genes
    genesUp = [father[:idxF], mother[:idxF]]
    genesDown = [father[idxF+1:], mother[idxM+1:]]
    
    #little bit mutation
    genesUp = mutation(genesUp, mutation_intensity)
    genesDown = mutation(genesDown, mutation_intensity)
    
    #new airfoil
    child1 = genesUp[0] + [0.0, 0.0] + genesDown[1]
    child2 = genesUp[1] + [0.0, 0.0] + genesDown[0]
    
    #make sure they are in good formate fist and last are [1.0, 0.0]
    child1[0] = child1[-1] = [1.0, 0.0]
    child2[0] = child2[-1] = [1.0, 0.0]
    
    return child1, child2

def mutation(gene, Kmu):
    pass

## script and setup

this is to setup the script and couple the python with Xfoil
    

__xfoil commend:__
    
    LOAD dat/e1211-il.dat	Load the dat file
    MDES	Go to the MDES menu
    FILT	Smooth any variations in the dat file data
    EXEC	Execute the smoothing
            Back to main menu
    PANE	Set the number and location of the airfoil points for analysis
    OPER	Go to the OPER menu
    ITER 70	Max number of iterations set to 70 for convergence
    RE 50000	Set Reynolds number (required?)
    VISC 50000	Set viscous calculation with Reynolds number
    PACC	Start polar output file
    polar/e1211-il_50000.txt	The output polar file name
            No dump file
    ALFA 0	Calculate lift and drag at 0° angle of attack
    ALFA 0.25	... 0.25°
    ALFA 0.5	... 0.5° ...
    ...	...more alpha calculations here ...
    ALFA 3.5	At 3.5° no convergence
    ALFA 3.5	... try again ...
    ALFA 3.5	... and again
    INIT	Run INIT to reinitialise
    ALFA 3.75	Skip to 3.75°
    ...	...rest of alpha calculations here ...
    PACC	Close polar file
    VISC	Reinitialise viscous calculation (required?)
    Down to main menu
    QUIT	Exit Xfoil

In [9]:
def script(airfoil_loc, airfoil, data_loc, AOT, RE):
    load_script = ['load '+airfoil_loc+airfoil+".txt", 
            airfoil,
            'MDES',
            'FILT',
            'EXEC',
            ' ',
            'PANE',
            'OPER',
            'ITER 150']       
    #define the output file name (different reynolds number has different file)
    data_file_name = data_loc+'/'+str(RE)+'.txt'
    #check and create file
    constant_set = ['RE ' + str(RE),
                    'VISC ' +str(RE),
                    'PACC', 
                    data_file_name,
                    ' ']
    #analyse different angle of attack
    alpha = ['ALFA '+str(a) for a in AOT]
    init = constant_set + alpha + ['PACC', ' ',]
    load_script += init
    load_script += [' ', ' ', 'QUIT\n']
    return load_script

In [10]:
def execute(airfoil_loc, airfoil, data_loc, angles, rey_range):
    #create a directory for each airfoil
    filedir = data_loc+airfoil
    os.makedirs(filedir)
    for RE in rey_range:
        #coupling xfoil into python
        ps = sp.Popen(['xfoil.exe'],
                      stdin=sp.PIPE,
                      stdout=None,
                      stderr=None,
                      encoding='utf8')
        #execute the script
        res = ps.communicate('\n'.join(script(airfoil_loc,airfoil, filedir, angle_range, RE)))
   

## run

This is to execute the whole programme with genetic algorithm.
    
    variables:
    epochs
    airfoil
    data_loc
    airfoil_loc
    reynolds number range
    angle range

In [11]:
#constants setting
epochs = 1
airfoil_dir = 'airfoil/'
data_dir = 'data/'
angle_range = frange(-10, 10, 0.25)
rey_range = range(100000, 600000, 100000)
num_parents = 2
mutation_intensity = 0.2

In [12]:
#reboost the system everytime it starts
boostup(data_dir)

In [20]:
#set generation
gen = '1gen'
#generate locations
data_loc = data_dir+gen+'/'
airfoil_loc = airfoil_dir+gen+'/'
# best airfoils that is going to produce children
winners = []
# those airfoils file location
winners_loc = {}

In [14]:
#execute airfoil to each of the airfoil in the folder
for airfoil in foil_mining('airfoil/1gen'):
    ang_range = list(angle_range)
    execute(airfoil_loc, airfoil, data_loc, ang_range+ang_range[::-1], rey_range)

In [21]:
mata_data = data_mining(data_loc, rey_range)

scores = sort_airfoil(heuristic(mata_data), winners)

# basically, the winners will be reused in the next generation for comparation see if the next generation got better
# choose the number of parents in this model
winners = scores[:num_parents]
#get the location in dict formate
winners_loc.update({i: airfoil_loc+i+'.txt' for i, _ in winners if i not in winners_loc})
winners_loc = {airfoil: winners_loc[airfoil] for airfoil, _ in winners}